# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/4944 [00:00<?, ?it/s]

In [ ]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [ ]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [ ]:
import torch
print(torch.version.cuda)

12.6


In [ ]:
# Instalación FAISS
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 22.6 MB/s eta 0:00:00


In [ ]:
# código base para FAISS
import faiss
import numpy as np

#Definición de query_embedding
# 1. Definir la consulta y generar su embedding
# Usamos la función embed_query que definiste antes
query_text = "What are the components of a battery?" # Puedes cambiar la pregunta
query_embedding = embed_query(query_text) # Esto genera el vector float32 de (1, D)

# 2. Configurar el índice FAISS (Asegúrate de que 'embeddings' ya terminó de cargar)
dimension = embeddings.shape[1]
# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 3. Realizar la búsqueda de los k=10 más cercanos
k = 10
D, I = index.search(query_embedding, k=10)

In [ ]:
# 4. Mostrar los resultados (índices encontrados)
print(f"Búsqueda finalizada para: '{query_text}'")
print("Índices de los chunks más similares:", I[0])
print("Distancias L2 correspondientes:", D[0])

Búsqueda finalizada para: 'What are the components of a battery?'
Índices de los chunks más similares: [47064 27366     3 47068 10543 59382 61361 72557 23514 72563]
Distancias L2 correspondientes: [0.33307645 0.34782708 0.35039032 0.35064596 0.35098594 0.3519713
 0.3520013  0.35365897 0.35417578 0.3545619 ]


## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.

In [ ]:
!pip install qdrant-client
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# Conectar en memoria
client = QdrantClient(":memory:")
COLLECTION_NAME = "wikipedia_collection"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 11.3 MB/s eta 0:00:00


2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)

In [ ]:
# 1. Crear la colección
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE),
)

# 2. Preparar los puntos (PointStruct) para la inserción masiva
# Usamos un generador para no saturar la RAM si el dataset es muy grande
points = [
    PointStruct(
        id=idx,
        vector=vector.tolist(),
        payload={
            "text": chunks_df.iloc[idx]["text"],
            "doc_id": int(chunks_df.iloc[idx]["doc_id"]),
            "chunk_id": int(chunks_df.iloc[idx]["chunk_id"])
        }
    )
    for idx, vector in enumerate(embeddings)
]

# 3. Insertar en lotes (upsert)
client.upsert(collection_name=COLLECTION_NAME, points=points)
print(f"Colección '{COLLECTION_NAME}' lista con {len(points)} puntos.")

Colección 'wikipedia_collection' lista con 79104 puntos.


/tmp/ipython-input-2851224880.py:23: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 79104 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client.upsert(collection_name=COLLECTION_NAME, points=points)


In [ ]:
!pip install -U qdrant-client

4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`


In [ ]:
# Esta función realiza la consulta y extrae los datos del payload de forma estructurada.
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# 1. Conexión limpia
client = QdrantClient(":memory:")
COLLECTION_NAME = "wiki_collection"

# 2. Re-creación e Inserción (Asegúrate de tener la matriz 'embeddings' lista)
if not client.collection_exists(COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE),
    )

    # Preparar puntos para la T4
    points = [
        PointStruct(
            id=i,
            vector=embeddings[i].tolist(),
            payload={
                "text": chunks_df.iloc[i]["text"],
                "doc_id": int(chunks_df.iloc[i]["doc_id"]),
                "chunk_id": int(chunks_df.iloc[i]["chunk_id"])
            }
        )
        for i in range(len(embeddings))
    ]
    client.upsert(collection_name=COLLECTION_NAME, points=points)

# 3. Función de búsqueda corregida (Entregable Parte 3)
def qdrant_search(query_embedding, k=5):
    # Convertir el embedding de la query a lista plana
    query_vector = query_embedding.flatten().tolist()

    # Usamos 'query_points' como alternativa robusta a 'search'
    response = client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=k,
        with_payload=True
    )

    # Mapeo de resultados
    return [
        (point.id, point.score, point.payload["text"],
         {"doc_id": point.payload["doc_id"], "chunk_id": point.payload["chunk_id"]})
        for point in response.points
    ]

# 4. Ejemplo de consulta
query_text = "Battery history and technology"
query_vec = embed_query(query_text)
resultados = qdrant_search(query_vec, k=5)

# 5. Salida formateada
print(f"Resultados para: {query_text}\n" + "="*50)
for res in resultados:
    print(f"ID: {res[0]} | Score: {res[1]:.4f} | Metadata: {res[3]}")
    print(f"Texto: {res[2][:160]}...\n")

Resultados para: Battery history and technology
ID: 39814 | Score: 0.8454 | Metadata: {'doc_id': 5426, 'chunk_id': 0}
Texto: Atomic battery The terms atomic battery, nuclear battery, tritium battery and radioisotope generator are used to describe a device which uses energy from the de...

ID: 28832 | Score: 0.8428 | Metadata: {'doc_id': 3952, 'chunk_id': 4}
Texto: he electrical energy carried on board a BEV to power the motors is obtained from a variety of battery chemistries arranged into battery packs. For additional ra...

ID: 72557 | Score: 0.8305 | Metadata: {'doc_id': 9996, 'chunk_id': 0}
Texto: List of battery sizes This article lists the sizes, shapes, and general characteristics of some common primary and secondary battery types in household and ligh...

ID: 15442 | Score: 0.8300 | Metadata: {'doc_id': 2083, 'chunk_id': 1}
Texto: rly well-suited to low-power electrical applications where long life of the energy source is needed, such as implantable medical devices or military 


### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`



### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.


In [ ]:
# 1. Definir la pregunta (query)
pregunta_usuario = "history of lead-acid batteries"

# 2. Generar embedding
query_vec = embed_query(pregunta_usuario)

# 3. Obtener resultados de Qdrant
k_resultados = 3
resultados = qdrant_search(query_vec, k=k_resultados)

# 4. Definir anchos fijos para las columnas
w_id = 6
w_score = 8
w_text = 60  # Ancho del resumen del texto

# 5. Imprimir encabezado con alineación exacta
print(f"Buscando: '{pregunta_usuario}'\n")

header = f"{'ID':<{w_id}} | {'SCORE':<{w_score}} | {'TEXTO (Resumen)':<{w_text}} | {'METADATA'}"
print(header)
print("-" * len(header))

# 6. Imprimir cada fila usando los mismos anchos
for res in resultados:
    id_punto, score, texto, metadata = res

    # Preparamos el resumen para que encaje exacto en el ancho w_text
    # Restamos 3 para los puntos suspensivos
    resumen = texto[:w_text-3].replace("\n", " ") + "..."

    # Formateamos la metadata
    meta_fmt = f"doc_id: {metadata['doc_id']}, chunk: {metadata['chunk_id']}"

    # Alineamos usando las variables de ancho
    # La sintaxis :<{w_id} asegura que el espacio ocupado sea siempre el mismo
    print(f"{str(id_punto):<{w_id}} | {score:<{w_score}.4f} | {resumen:<{w_text}} | {meta_fmt}")

Buscando: 'history of lead-acid batteries'

ID     | SCORE    | TEXTO (Resumen)                                              | METADATA
-------------------------------------------------------------------------------------------
75903  | 0.8325   | normally used lead acid batteries to store a full 120-vol... | doc_id: 10377, chunk: 2
15442  | 0.8314   | rly well-suited to low-power electrical applications wher... | doc_id: 2083, chunk: 1
71872  | 0.8293   | is achieved. Accepted average float voltages for lead-aci... | doc_id: 9888, chunk: 2



### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?

Como se está usando el modelo de embeddings E5-base-v2, se utilizó la métrica Coseno (Distance.COSINE), además de que, al haber generado los vectores con la opción normalize_embeddings=True, éstos se encuentran en un espacio vectorial normalizado donde la dirección del vector es más relevante que su magnitud para determinar la similitud semántica.

- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?

A diferencia de FAISS (donde la metadata es externa al índice lo que obliga a mantener un dataframe separado y mapear manualmente los índices para recuperar el texto)
Qdrant integra la metadata directamente en el payload del vector, permitiendo que la búsqueda y la recuperación de información se realicen en una única operación atómica.

- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?

El tiempo de respuesta aumenta de forma marginal o despreciable para valores típicos de k


## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.

In [ ]:
!pip install "pymilvus[milvus_lite]"
from pymilvus import MilvusClient, DataType
import time

# Conexión local (crea un archivo .db en el directorio actual)
client_milvus = MilvusClient("milvus_wikipedia.db")
COLLECTION_NAME = "wiki_vectors"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 21.5 MB/s eta 0:00:00


2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)


In [ ]:
import os
import signal
import gc

# 1. Eliminar el objeto cliente si existe
if 'client_milvus' in locals():
    del client_milvus

# 2. Forzar recolección de basura
gc.collect()

# 3. Borrar archivos de socket y base de datos
# Milvus Lite crea archivos en /tmp que a veces causan el "Connection refused"
!rm -rf /tmp/milvus* !rm -f milvus_wikipedia.db*

print("Entorno de Milvus limpiado. Procediendo a reinstalación de seguridad.")

Entorno de Milvus limpiado. Procediendo a reinstalación de seguridad.


In [ ]:
from pymilvus import MilvusClient
import time

# 1. Nueva conexión con nombre de archivo diferente para evitar bloqueos
client_milvus = MilvusClient("milvus_wikipedia_v2.db")
COLLECTION_NAME = "wiki_vectors_fixed"



In [ ]:
# 2. Crear colección (simplificada)
client_milvus.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=embeddings.shape[1],
    metric_type="COSINE"
)

3. Insertar `N` embeddings.

In [ ]:


# 3. Insertar una muestra controlada (10,000 vectores)
# Esto es suficiente para comparar FLAT vs HNSW
limite = 10000
data_batch = [
    {
        "id": i,
        "vector": embeddings[i].tolist(),
        "text": chunks_df.iloc[i]["text"],
        "doc_id": int(chunks_df.iloc[i]["doc_id"])
    }
    for i in range(limite)
]

client_milvus.insert(collection_name=COLLECTION_NAME, data=data_batch)
print(f"✅ {len(data_batch)} vectores insertados con éxito.")


✅ 10000 vectores insertados con éxito.


4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).

In [ ]:

# 4. Crear índice ANN
# CONFIGURACIÓN 1: Búsqueda Exacta (FLAT)
# FLAT es el índice por defecto para precisión máxima
index_params_exact = client_milvus.prepare_index_params()
index_params_exact.add_index(
    field_name="vector",
    index_type="FLAT",
    metric_type="COSINE"
)
client_milvus.create_index(COLLECTION_NAME, index_params_exact)

# CONFIGURACIÓN 2: Búsqueda ANN (IVF_FLAT)
# Usamos IVF_FLAT ya que HNSW no es compatible con Milvus Lite
index_params_ann = client_milvus.prepare_index_params()
index_params_ann.add_index(
    field_name="vector",
    index_type="IVF_FLAT",
    metric_type="COSINE",
    params={"nlist": 128} # nlist es el número de clusters para la búsqueda
)
client_milvus.create_index(COLLECTION_NAME, index_params_ann)

# Cargar la colección para habilitar la búsqueda
client_milvus.load_collection(COLLECTION_NAME)

5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

In [ ]:
# FUNCIÓN DE BÚSQUEDA
def milvus_search(query_embedding, k=5):
    q_vec = query_embedding.flatten().tolist()
    start = time.time()
    res = client_milvus.search(
        collection_name=COLLECTION_NAME,
        data=[q_vec],
        limit=k,
        output_fields=["text", "doc_id"],
        # nprobe define cuántos clusters revisar; a más nprobe, más precisión
        search_params={"metric_type": "COSINE", "params": {"nprobe": 10}}
    )
    return res[0], (time.time() - start)

In [ ]:
# Pruebas
query_text = "how lead acid batteries are recycled"
query_vec = embed_query(query_text)

print(f"Resultados del experimento (IVF_FLAT) para: '{query_text}'")
for val_k in [5, 20]:
    hits, latencia = milvus_search(query_vec, k=val_k)
    print(f"\n[k={val_k}] Latencia: {latencia:.6f}s")
    print(f"Mejor ID: {hits[0]['id']} | Texto: {hits[0]['entity']['text'][:80]}...")

Resultados del experimento (IVF_FLAT) para: 'how lead acid batteries are recycled'

[k=5] Latencia: 0.024683s
Mejor ID: 1858 | Texto: the world in favor for new ones. About 70 percent of overall toxic in landfills ...

[k=20] Latencia: 0.006655s
Mejor ID: 1858 | Texto: the world in favor for new ones. About 70 percent of overall toxic in landfills ...




### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?

Para el parámetro de índice se definió nlist=128. Este parámetro determina en cuántos "clústeres" o regiones se divide el espacio vectorial.
Para el de búsqueda se utilizó nprobe=10 en los search_params, como el control crítico que define cuántos de esos clústeres se revisarán durante la consulta; que para nprobe cercano a 1 siendo ultra rápido pero con riesgo de perder el vecino más cercano real si este cayó en el borde de otro clúster y cercano a 10, se revisa múltiples regiones, aumentando la probabilidad de encontrar el resultado exacto a costa de una mayor latencia.



- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?

La evidencia más clara se obtiene al comparar la lista de IDs devuelta por el índice FLAT (exacto) contra la de IVF_FLAT (ANN), puesto de que ANN es un método probabilístico que agrupa vectores, a veces puede ignorar un vector que está físicamente más cerca pero "oculto" en un clúster que el algoritmo decidió no explorar, lo que resulta en una lista de resultados "aproximada" en lugar de la lista exhaustiva garantizada por FLAT.


## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.

In [ ]:
# Instalación y Conexión
# 1. Instalar weaviate-client y un servidor embebido si es necesario
!pip install -U weaviate-client

import weaviate
import os

# 2. Conexión a instancia embebida
# Esto descargará el binario de Weaviate y lo ejecutará localmente en el notebook
client_weaviate = weaviate.connect_to_embedded()

print("¡Conexión exitosa a Weaviate Embebido!")

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.30.5/weaviate-v1.30.5-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 25032


¡Conexión exitosa a Weaviate Embebido!



2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)

In [ ]:
import weaviate.classes.config as wvcc

# 1. Definir el nombre de la colección
COLLECTION_NAME = "Document"

# 2. Eliminar la colección si ya existía de intentos previos para evitar errores
if client_weaviate.collections.exists(COLLECTION_NAME):
    client_weaviate.collections.delete(COLLECTION_NAME)

# 3. Crear la colección con sus propiedades
client_weaviate.collections.create(
    name=COLLECTION_NAME,
    vectorizer_config=None,  # "None" porque los embeddings ya los creamos en la Parte 1
    properties=[
        wvcc.Property(name="text", data_type=wvcc.DataType.TEXT),
        wvcc.Property(name="doc_id", data_type=wvcc.DataType.INT),
        wvcc.Property(name="chunk_id", data_type=wvcc.DataType.INT),
        # Puedes añadir más aquí, por ejemplo:
        # wvcc.Property(name="title", data_type=wvcc.DataType.TEXT),
    ]
)

print(f"Esquema definido: La colección '{COLLECTION_NAME}' está lista.")

Esquema definido: La colección 'Document' está lista.



3. Insertar objetos con:
   - propiedades + vector

In [ ]:
# 1. Obtener la referencia a la colección creada en el paso anterior
documents = client_weaviate.collections.get("Document")

# 2. Configurar la inserción por lotes (Batch)
# Usaremos una muestra de 20,000 para garantizar que el notebook se mantenga estable
limite_insercion = 20000

print(f"Insertando {limite_insercion} objetos en Weaviate...")

with documents.batch.dynamic() as batch:
    for i in range(limite_insercion):
        batch.add_object(
            properties={
                "text": chunks_df.iloc[i]["text"],
                "doc_id": int(chunks_df.iloc[i]["doc_id"]),
                "chunk_id": int(chunks_df.iloc[i]["chunk_id"])
            },
            vector=embeddings[i].tolist()  # Conversión obligatoria a lista
        )

# 3. Validar la carga
# En Weaviate v4, podemos usar aggregate para contar los objetos rápidamente
total = documents.aggregate.over_all(total_count=True)
print(f"Éxito: Se han insertado {total.total_count} objetos en la colección.")

Insertando 20000 objetos en Weaviate...
Éxito: Se han insertado 20000 objetos en la colección.


4. Consultar por similitud (Top-k) con `query_embedding`.

In [ ]:
# 1. Definir la query y generar el embedding (usando tu función de la Parte 1)
query_text = "How does a lead-acid battery store energy?"
query_vec = embed_query(query_text)

# 2. Realizar la búsqueda por similitud vectorial
# Weaviate requiere que el vector sea una lista plana
results = documents.query.near_vector(
    near_vector=query_vec.flatten().tolist(),
    limit=5,
    return_metadata=weaviate.classes.query.MetadataQuery(distance=True)
)

# 3. Mostrar los resultados
print(f"Resultados para: '{query_text}'\n")
for i, obj in enumerate(results.objects):
    distancia = obj.metadata.distance
    texto = obj.properties['text']
    meta = f"Doc: {obj.properties['doc_id']}, Chunk: {obj.properties['chunk_id']}"

    print(f"{i+1}. [Distancia: {distancia:.4f}]")
    print(f"   Metadata: {meta}")
    print(f"   Contenido: {texto[:150]}...\n")

Resultados para: 'How does a lead-acid battery store energy?'

1. [Distancia: 0.1701]
   Metadata: Doc: 993, Chunk: 19
   Contenido: eded. This DC power is supplied by the battery pack, and the controller regulates the power to the motor, supplying either variable pulse width DC or ...

2. [Distancia: 0.1711]
   Metadata: Doc: 1731, Chunk: 0
   Contenido: Thermal Battery A thermal energy battery is a physical structure used for the purpose of storing and releasing thermal energyâ€”see also thermal energ...

3. [Distancia: 0.1771]
   Metadata: Doc: 1123, Chunk: 5
   Contenido: battery. Heat pumps with hot water storage and electric vehicles have been found to have higher potential on reduction of emissions and fossil fuel us...

4. [Distancia: 0.1783]
   Metadata: Doc: 1731, Chunk: 1
   Contenido: py of fusion or delta enthalpy of vaporization. Thermal batteries are very common, and include such familiar items as a hot water bottle. Early exampl...

5. [Distancia: 0.1785]
   Metadata: D

5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

In [ ]:
import weaviate.classes.query as wvq

def weaviate_search(query_embedding, k=5):
    """
    Realiza una búsqueda semántica en Weaviate y devuelve resultados formateados.
    """
    # Convertir el embedding a lista plana (formato requerido por Weaviate)
    query_vector = query_embedding.flatten().tolist()

    # Ejecutar la consulta vectorial
    response = documents.query.near_vector(
        near_vector=query_vector,
        limit=k,
        return_metadata=wvq.MetadataQuery(distance=True)
    )

    formatted_results = []

    for obj in response.objects:
        # Extraer los datos requeridos
        res_id = obj.uuid  # ID único (UUID) generado por Weaviate
        score = obj.metadata.distance  # Distancia como métrica de score
        text = obj.properties.get("text")

        # Agrupar el resto de la metadata
        metadata = {
            "doc_id": obj.properties.get("doc_id"),
            "chunk_id": obj.properties.get("chunk_id")
        }

        formatted_results.append((res_id, score, text, metadata))

    return formatted_results

# --- Ejemplo de ejecución del entregable ---
query_text = "What is the history of batteries?"
query_vec = embed_query(query_text)
resultados = weaviate_search(query_vec, k=3)

print(f"Resultados de la Función Entregable:\n" + "="*50)
for res in resultados:
    uid, dist, txt, meta = res
    print(f"ID: {uid}")
    print(f"Score (Distancia): {dist:.4f}")
    print(f"Metadata: {meta}")
    print(f"Texto: {txt[:120]}...\n" + "-"*30)

Resultados de la Función Entregable:
ID: 4d943c59-f3c8-4820-91f3-e286c51b1a70
Score (Distancia): 0.1982
Metadata: {'doc_id': 1534, 'chunk_id': 0}
Texto: Timeline of hydrogen technologies This is a timeline of the history of hydrogen technology....
------------------------------
ID: a52ca2d7-5a8c-4d37-bb2a-3545220b29c0
Score (Distancia): 0.1983
Metadata: {'doc_id': 909, 'chunk_id': 0}
Texto: Gravity battery A gravity battery is a type of mechanical battery that stores gravitational potential energy, by raising...
------------------------------
ID: b2011dd7-1d8d-4917-8950-cbd699142eee
Score (Distancia): 0.1990
Metadata: {'doc_id': 2083, 'chunk_id': 1}
Texto: rly well-suited to low-power electrical applications where long life of the energy source is needed, such as implantable...
------------------------------


### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?

La "tabla + filas" (Milvus/FAISS) es un modelo relacional/plano donde los datos son registros en una estructura rígida, mientras el modelo de "schema + objetos" de Weaviate es orientado a entidades lo cual permite definir clases que representan conceptos del mundo real con varios tipos de datos (como relaciones semánticas), funcionando más como una base de datos de grafos que como una hoja de cálculo.

- ¿Cómo describirías el trade-off de complejidad vs expresividad?

Weaviate requiere una mayor configuración inicial, ya que hay que definir tipos de datos, clases y configuraciones de vectorización; sin embargo, esta complejidad se paga con una altísima expresividad, ya que permite realizar consultas híbridas (vectorial + texto + filtros) en un lenguaje muy natural y manejar metadatos complejos sin perder rendimiento.


## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.

In [ ]:
# Primero vamos a la instalación y configuración

!pip install chromadb

import chromadb
import time

# 1. Crear el cliente y la colección
# Usamos el cliente efímero (en memoria) para prototipado rápido
chroma_client = chromadb.Client()
collection_chroma = chroma_client.create_collection(name="wiki_collection")

2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)

In [ ]:
# 1. Preparar las listas de datos
all_embeddings = embeddings.tolist()
all_documents = chunks_df['text'].tolist()
all_metadatas = [{"doc_id": int(d), "chunk_id": int(c)}
                 for d, c in zip(chunks_df['doc_id'], chunks_df['chunk_id'])]
all_ids = [str(i) for i in range(len(chunks_df))]

# 2. Configurar el tamaño del lote
batch_size = 5000
total_len = len(all_ids)

print(f"Iniciando inserción de {total_len} documentos en lotes de {batch_size}...")

# 3. Bucle de inserción por lotes
for i in range(0, total_len, batch_size):
    end_idx = min(i + batch_size, total_len)

    collection_chroma.add(
        embeddings=all_embeddings[i:end_idx],
        documents=all_documents[i:end_idx],
        metadatas=all_metadatas[i:end_idx],
        ids=all_ids[i:end_idx]
    )
    print(f"Progreso: {end_idx}/{total_len} documentos insertados.")

print(f"\nÉxito final: {collection_chroma.count()} documentos indexados en Chroma.")

Iniciando inserción de 79104 documentos en lotes de 5000...
Progreso: 5000/79104 documentos insertados.
Progreso: 10000/79104 documentos insertados.
Progreso: 15000/79104 documentos insertados.
Progreso: 20000/79104 documentos insertados.
Progreso: 25000/79104 documentos insertados.
Progreso: 30000/79104 documentos insertados.
Progreso: 35000/79104 documentos insertados.
Progreso: 40000/79104 documentos insertados.
Progreso: 45000/79104 documentos insertados.
Progreso: 50000/79104 documentos insertados.
Progreso: 55000/79104 documentos insertados.
Progreso: 60000/79104 documentos insertados.
Progreso: 65000/79104 documentos insertados.
Progreso: 70000/79104 documentos insertados.
Progreso: 75000/79104 documentos insertados.
Progreso: 79104/79104 documentos insertados.

Éxito final: 79104 documentos indexados en Chroma.



3. Consultar Top-k con `query_embedding`.


In [ ]:
def chroma_search(query_embedding, k=5):
    """
    Realiza una búsqueda semántica en ChromaDB.
    """
    # Chroma permite pasar el vector directamente como lista
    query_vector = query_embedding.flatten().tolist()

    start_time = time.time()
    results = collection_chroma.query(
        query_embeddings=[query_vector],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )
    latency = time.time() - start_time

    # Formatear la salida para el usuario
    formatted_results = []
    for i in range(len(results['ids'][0])):
        formatted_results.append({
            "id": results['ids'][0][i],
            "score": results['distances'][0][i],
            "text": results['documents'][0][i],
            "metadata": results['metadatas'][0][i]
        })

    return formatted_results, latency

# --- EJECUCIÓN DEL ENTREGABLE (Consulta con k=5) ---
query_text = "how much lead is in a typical battery?"
query_vec = embed_query(query_text)

res_chroma, t_chroma = chroma_search(query_vec, k=5)

print(f"Resultados de Chroma (Latencia: {t_chroma:.4f}s) para: '{query_text}'\n" + "-"*80)
for r in res_chroma:
    print(f"ID: {r['id']} | Distancia: {r['score']:.4f}")
    print(f"Meta: {r['metadata']}")
    print(f"Texto: {r['text'][:120]}...\n")

Resultados de Chroma (Latencia: 0.0175s) para: 'how much lead is in a typical battery?'
--------------------------------------------------------------------------------
ID: 46415 | Distancia: 0.2676
Meta: {'chunk_id': 36, 'doc_id': 6358}
Texto: cies ranged from 91% to 94.5%, depending on the batteryâ€™s state of charge. [REF] This is compared with a Sandia Nation...

ID: 71872 | Distancia: 0.2928
Meta: {'chunk_id': 2, 'doc_id': 9888}
Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in following table: Compensa...

ID: 72562 | Distancia: 0.3025
Meta: {'doc_id': 9996, 'chunk_id': 5}
Texto: dmium) and NiMH (nickel metal hydride) typically output 1.25Â volts per cell. Devices intended for use with primary batt...

ID: 72559 | Distancia: 0.3088
Meta: {'chunk_id': 2, 'doc_id': 9996}
Texto: tituting a battery. The current IEC standards for portable primary (non-rechargeable) batteries bear the 60086 number. T...

ID: 59383 | Distancia: 0.3088


### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?

A diferencia de Milvus, que requirió lidiar con esquemas rígidos, tipos de datos de gRPC y el colapso del socket local, o Qdrant, que necesitó una estructura de "puntos" específica, Chroma permite una integración casi "plana". La única fricción real fue el límite de batch size de 5461, que es una restricción del motor de Rust de Chroma, pero que se resuelve fácilmente con un bucle simple. No hubo necesidad de configurar parámetros complejos de índices (como HNSW o IVF) para que funcionara de inmediato.

- ¿Qué limitaciones ves para un sistema en producción?

Debo seguir provando para dar una respuesta más específica... pero de lo que noté en cuanto a la gestión de recursos, al operar principalmente en memoria o archivos locales simples, no gestiona tan bien los millones de vectores como lo hace la arquitectura distribuida de Milvus.

## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.

In [ ]:
# 1. Instalar dependencias de desarrollo para Postgres 14
!apt-get update
!apt-get -y install postgresql-server-dev-14 build-essential

# 2. Clonar y compilar pgvector desde el código fuente
!git clone https://github.com/pgvector/pgvector.git
%cd pgvector
!make
!make install
%cd ..

# 3. Reiniciar el servicio de Postgres para asegurar que reconozca los nuevos archivos
!service postgresql restart

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,227 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,573 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu j

In [ ]:
# Instalación y Configuración
# Instalación de dependencias
!apt-get -y install postgresql-14
!service postgresql start
!sudo -u postgres psql -c "CREATE USER colab WITH PASSWORD 'colab';"
!sudo -u postgres psql -c "CREATE DATABASE vector_db OWNER colab;"
!sudo -u postgres psql -d vector_db -c "CREATE EXTENSION IF NOT EXISTS vector;"

# Librería de Python para interactuar con pgvector
!pip install pgvector psycopg2-binary

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libllvm14
  libtypes-serialiser-perl logrotate netbase postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libllvm14
  libtypes-serialiser-perl logrotate netbase postgresql-14
  postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 41 not upgraded.
Need to get 42.4 MB of archives.
After this operation, 161 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 netbase all 6.3

2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)

In [ ]:
# 1. Elevar al usuario 'colab' a superusuario desde la terminal
!sudo -u postgres psql -c "ALTER USER colab WITH SUPERUSER;"

print("Privilegios de Superusuario otorgados a 'colab'.")

ALTER ROLE
Privilegios de Superusuario otorgados a 'colab'.


In [ ]:
import psycopg2
from pgvector.psycopg2 import register_vector

# 1. Conexión
conn = psycopg2.connect(
    dbname="vector_db",
    user="colab",
    password="colab",
    host="localhost"
)
cur = conn.cursor()

# 2. ACTIVACIÓN (Ahora los archivos .control ya están en su lugar)
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()

# 3. Registro del tipo en Python
register_vector(conn)

# 4. Creación de la tabla
dimension = embeddings.shape[1]
cur.execute("DROP TABLE IF EXISTS documents;")
cur.execute(f"""
    CREATE TABLE documents (
        id SERIAL PRIMARY KEY,
        text TEXT,
        doc_id INTEGER,
        chunk_id INTEGER,
        embedding vector({dimension})
    );
""")
conn.commit()

print(f"Tabla creada con éxito en PostgreSQL.")

Tabla creada con éxito en PostgreSQL.


3. Insertar todos los documentos y embeddings.

In [ ]:
from psycopg2.extras import execute_batch

# 1. Preparar los datos en el formato que espera la consulta SQL
# Combinamos texto, IDs y el embedding (que pgvector acepta como lista o array)
print("Preparando datos para la inserción...")
data_to_insert = [
    (
        chunks_df.iloc[i]["text"],
        int(chunks_df.iloc[i]["doc_id"]),
        int(chunks_df.iloc[i]["chunk_id"]),
        embeddings[i].tolist() # Convertimos a lista para compatibilidad total
    )
    for i in range(len(embeddings))
]

# 2. Definir la consulta SQL
insert_query = """
    INSERT INTO documents (text, doc_id, chunk_id, embedding)
    VALUES (%s, %s, %s, %s);
"""

# 3. Ejecutar la inserción por lotes
batch_size = 2000 # Tamaño de lote balanceado para memoria y velocidad
print(f"Insertando {len(data_to_insert)} registros en PostgreSQL...")

try:
    execute_batch(cur, insert_query, data_to_insert, page_size=batch_size)
    conn.commit()
    print("Inserción masiva completada exitosamente.")
except Exception as e:
    conn.rollback()
    print(f"Error durante la inserción: {e}")

# 4. Verificar el conteo final
cur.execute("SELECT COUNT(*) FROM documents;")
count = cur.fetchone()[0]
print(f"Total de filas en la tabla 'documents': {count}")

Preparando datos para la inserción...
Insertando 79104 registros en PostgreSQL...
Inserción masiva completada exitosamente.
Total de filas en la tabla 'documents': 79104


4. Consultar Top-k por similitud, ordenando por distancia.


In [ ]:
# Resetear la transacción si hay falla
conn.rollback()
print("Transacción reseteada. La conexión vuelve a estar limpia.")

Transacción reseteada. La conexión vuelve a estar limpia.


In [ ]:
def pgvector_search(query_embedding, k=5):
    """
    Realiza una búsqueda semántica con cast explícito para evitar errores de tipo.
    """
    query_vec = query_embedding.flatten().tolist()

    # IMPORTANTE: El %s::vector es la clave para que no falle
    search_query = """
    SELECT
        id,
        embedding <=> %s::vector AS score,
        text,
        doc_id,
        chunk_id
    FROM documents
    ORDER BY score ASC
    LIMIT %s;
    """

    try:
        cur.execute(search_query, (query_vec, k))
        rows = cur.fetchall()

        formatted_results = []
        for row in rows:
            formatted_results.append({
                "id": row[0],
                "score": row[1],
                "text": row[2],
                "metadata": {"doc_id": row[3], "chunk_id": row[4]}
            })
        return formatted_results
    except Exception as e:
        conn.rollback() # Si falla, limpiamos automáticamente para la próxima
        print(f"Error en la consulta: {e}")
        return []

# --- PRUEBA FINAL ---
query_text = "how lead acid batteries are recycled"
query_vec = embed_query(query_text)
resultados_pg = pgvector_search(query_vec, k=5)

if resultados_pg:
    print(f"Búsqueda exitosa en PostgreSQL:\n" + "="*50)
    for res in resultados_pg:
        print(f"Score: {res['score']:.4f} | Texto: {res['text'][:100]}...")

Búsqueda exitosa en PostgreSQL:
Score: 0.1433 | Texto: :2004 certification standards. UltraBatteryâ€™s electrolyte solution contains H2SO4 in water, and it...
Score: 0.1562 | Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...
Score: 0.1638 | Texto: e indicates that the batteryâ€™s entire capacity would be used (or replaced if charging) in one hour...
Score: 0.1638 | Texto: tteries â€œfrom zero SOC to 84% SOC the average overall battery charging efficiency is 91%â€. While...
Score: 0.1648 | Texto: mon to the lead acid cell and the ultracapacitor. This technology (specifically the addition of the ...



### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?

Es la más explicable de todas, pues mientras que en Milvus o Weaviate la búsqueda ocurre en una "caja negra" accesible solo vía API o SDKs específicos, en pgvector la búsqueda es código SQL estándar.

- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?

Esta es su mayor fortaleza. En una base vectorial dedicada, hacer un filtro complejo suele ser limitado, y aunque si se me ha complicado administrar bases de datos con SQL, hay varias ventajas en lo que puedes hacer, como cruzar tus vectores con tablas de usuarios, ventas o logs históricos de forma nativa, o usar filtros complejos para combinar la búsqueda semántica con condiciones relacionales; en general, como ya conozco el uso de queries para realizar consultas con SQL, resulta más fácil esa parte.

- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?

El consumo de RAM, ya que PostgreSQL carga los índices en su buffer cache; en una base de datos de miles de millones de vectores, el índice va a desplazar los datos relacionales de la memoria, degradando el rendimiento general de la base de datos (de hecho, tuve que reiniciar la sesión del colab 3 veces debido a el consumo total de la RAM).